### 1

В каких задачах честный A/B эксперимент может быть невозможно провести?

- [+] Тестирование нового алгоритма скидок на товары, близкие к концу срока годности
- [+] Тестирование нового алгоритма ценообразования в сервисе такси
- Тестирование нового алгоритма генерации музыкальных плейлистов
- Тестирование нового алгоритма поиска по тексту

**Комментарий**

В поиске и музыкальном сервисе всё заметно проще — пользователи друг на друга не влияют, провести честный A/B можно.

В случае тестирования скидок на товары есть риск, что тестовая группа с новым алгоритмом скидок быстрее раскупит товар и люди в контрольной группе тоже не смогут его купить. Метрика "число списаний" будет посчитана некорректно.

В случае нового алгоритма ценообразования в сервисе такси опять же пользователь с более низкой ценой может заказать машину, а пользователю в контрольной группе эта машина не достанется.

### 2

В чём сложность проведения A/B экспериментов с two-sided маркетплейсами?

- С two-sided маркетплейсами можно проводить A/B эксперименты, а вот three-sided и более уже технически невозможно
- [+] Группы любой из сторон маркетплейса после разбиения не получаются независимыми, так как связаны через вторую сторону маркетплейса
- Сложно обеспечить равномерное разбиение одной из сторон маркетплейса

**Комментарий**

Одно из основных требований в A/B экспериентах – независимость наблюдений. В случае двустороннего маркетплейса группа тестовых пользователей через вторую сторону маркетплейса оказывает эффект на контрольную группу.

### 3

В чём идея switchback-экспериментов?

- [+] Вместо пользователей в эксперименте разбивается на группы время, каждое из которых соответствует либо контрольной, либо тестовой выборке
- Мы разбиваем пользователей на группы как в обычном A/B, но соль эксперимента меняем каждый день
- Мы вычитаем эффект от связи групп в A/B через синтетический контроль, по которому оцениваем размер перетекания

**Комментарий**

Идея switchback эксперимента как раз в том, чтобы мы включали тестовую группу на всех (при этом влияем на обе стороны маркетплейса), исключая эффект влияния групп друг на друга, как если бы запускали A/B. То есть мы на самом деле делим на контрольную и тестовую группы время нашего эксперимента.

Если бы мы разбивали пользователей как в A/B, но меняли соль, то у нас получилось бы что-то странное — группы всё-таки есть, они влияют друг на друга из-за второй стороны маркетплейса.

### 4

Представим, что мы разрабатываем маркетплейс, то есть объединяем продавцов и покупателей товаров. 

Наша аудитория за период эксперимента 5 миллионов пользователей, каждый из которых сделал 1 покупку, 5000 продавцов. Switchback-эксперимент длился 2 недели, мы равномерно делили 50/50, переключали алгоритмы каждый час. Наша целевая метрика «число успешных покупок». Метрики мы хотим считать по unit-ам.

Сколько наблюдений будет в каждой нашей выборке для анализа значимости (то есть либо тестовой, либо контрольной)?

- 2500
- 1000
- 2500000
- 6250000000
- [+] 168

**Комментарий**

Так как мы разбиваем по часам, то за 2 недели у нас будет 2*7*24 = 336 часов, половина из которых (168) попадёт в контрольную или тестовую выборки.

### 5

В каких из примеров нельзя провести switchback-эксперимент или A/B эксперимент?

- Ранжирование постов в социальной сети
- [+] Появление дизлайков к твитам
- Алгоритм ценообразования в сервисе такси

**Комментарий**

Чаще всего это примеры социальных сервисов, где довольно заметно, что появилась новая функциональность, поэтому нельзя её дать только части пользователей.

### 6

Идея синтетического контроля в том, чтобы использовать модели. Представим, что у нас сервис объявлений, работающий в разных городах страны. Мы хотим проверить, что новая функциональность взаимодействия пользователей с объявлениями даст улучшение метрик. Для этого используем технику синтетического контроля и запускаем эксперимент в Самаре и Волгограде.

Сопоставьте пары, что чем является в эксперимент и при обучении модели.

- Признаки обучающей выборки - Значения метрик до запуска на других городах
- Признаки тестовой выборки - Значения метрик после запуска на других городах
- Таргет обучающей выборки - Значения метрик до запуска на городах запуска
- Предсказания тестовой выборки - Значения синтетического контроля

### 7

Код сервиса можно посмотреть отдельно. Обратите внимание на функцию для разбиения пользователей и как она используется. Также теперь в решении есть развлетвление на то, какую модель использовать.

### 8

Загрузим данные и посмотрим на них

In [1]:
import numpy as np
import pandas as pd

views = pd.read_csv('views.csv')
likes = pd.read_csv('likes.csv')

In [2]:
likes.head()

,user_id,post_id,timestamp
0,128381,4704,1654030804
1,146885,1399,1654030816
2,50948,2315,1654030828
3,14661,673,1654030831
4,37703,1588,1654030833


In [3]:
views.head()

,user_id,exp_group,recommendations,timestamp
0,128381,control,[3644 4529 4704 5294 4808],1654030803
1,146885,test,[1399 1076 797 7015 5942],1654030811
2,50948,test,[2315 3037 1861 6567 4093],1654030825
3,37703,test,[2842 1949 162 1588 6794],1654030826
4,14661,test,[2395 5881 5648 3417 673],1654030829


### 9

Начнём с проверки разбиения групп.

Во-первых, у нас нет таблички соответствия пользователь-группа, так как мы на самом деле определяли группу пользователя прямо перед подготовкой рекомендаций.

Может показаться, что это гарантирует нам однозначное соответствие групп для пользователей, но давайте это проверим. В реальности бывают лаги системы (например, если мы запрашиваем группу пользователя, то не всегда можем получить ответ) и это свойство не выполняется. Проверьте, нет ли у нас пользователей, которые попали в обе группы. Если их совсем немного, удалите их из обеих выборок (если бы было много, то надо было разбираться, что пошло не так).

Теперь, когда мы знаем про однозначное соответствие, сделайте табличку пользователь-группа. Проверьте, что группы получаются одинакового размера по пользователям. Для этого можно посчитать долю каждой группы, а ещё можно применить критерий для долей (биномиальный тест), чтобы проверить соответствие этой доли ожидаемым 0.5.

- Все пользователи соответствуют только 1-й группе, группы неравного размера.
- Все пользователи соответствуют только 1-й группе, группы равного размера.
- Есть много пользователей сразу в 2 группах, группы равного размера.
- [+] Есть немного пользователей сразу в 2 группах, группы равного размера.

**Решение**

In [4]:
tmp = views.groupby('user_id').exp_group.nunique().reset_index()
tmp[tmp.exp_group > 1]

,user_id,exp_group
10071,25623,2
20633,55788,2
54475,142283,2
57065,148670,2


Всего 4 пользователя, удалим их

In [5]:
bad_users = tmp[tmp.exp_group > 1].user_id.values

views = views[~np.in1d(views.user_id, bad_users)]
likes = likes[~np.in1d(likes.user_id, bad_users)]

Проверим, что группы разбились нормально

In [6]:
views.groupby('user_id').first().exp_group.value_counts(normalize='True')

test       0.502377
control    0.497623
Name: exp_group, dtype: float64

Похоже на правду, но проверим критерием

In [7]:
views.groupby('user_id').first().exp_group.value_counts()

test       32659
control    32350
Name: exp_group, dtype: int64

In [8]:
from scipy.stats import binomtest

binomtest(k=32659, n=32659+32350, p=0.5)

BinomTestResult(k=32659, n=65009, alternative='two-sided', proportion_estimate=0.5023765940100602, pvalue=0.2270501563614752)

Кажется, что всё хорошо

### 10

Теперь давайте попробуем оценить, а улучшаются ли наши метрики в тестовой группе.

Наши данные о показах и кликах хранятся в разных табличках и просто так их не получается сджойнить. Давайте оставим это на потом, а пока попробуем оценить более простыми метриками. Подумайте, какие метрики можно было бы посчитать без джойна показов и кликов.

Соберите табличку, в которой будут пользователи, попавшие в наш эксперимент (то есть те, для которых мы строили хотя бы одну рекомендацию). Посчитайте, кто из них сколько лайков сделал. Наверняка будут пользователи, которые не сделали ни один лайк.

Посчитайте долю пользователей, которая сделала хотя бы один лайк за время эксперимента без разбивки на группы.

- 33.6%
- 23.8%
- [+] 89.5%
- 28.2%

**Решение**

Соберём разбиение на группы

In [9]:
groups = views.groupby('user_id').first().reset_index()[['user_id', 'exp_group']]
groups.head()

,user_id,exp_group
0,200,test
1,201,test
2,202,test
3,212,test
4,213,test


Посчитаем число лайков на пользователя

In [10]:
users_w_likes = (
    likes
    .groupby('user_id')
    .post_id.count()
    .reset_index()
    .rename(columns={'post_id': 'like_num'})
)
users_w_likes.head()

,user_id,like_num
0,200,1
1,201,3
2,202,2
3,212,4
4,213,7


Соберём вместе

In [11]:
tmp = pd.merge(groups, users_w_likes, on='user_id', how='left')
tmp.like_num = tmp.like_num.fillna(0)
tmp['has_like'] = (tmp['like_num'] > 0).astype(int)

In [12]:
tmp.groupby('exp_group')[['has_like', 'like_num']].mean()

,has_like,like_num
exp_group,,
control,0.891314,3.487079
test,0.898160,3.592578


Здесь мы посчитали отдельно по группам, но уже понятно, какой ответ нам подходит

### 11

А теперь давайте оценим, различаются ли число лайков между группами. Выберите подходящий статистический критерий и проверьте две метрики: доля пользователей с хотя бы одним лайком, число лайков на пользователя. 

Какой статистические критерий наиболее хорошо подходит для метрики "число лайков на пользователя"?

- z-критерий для долей
- [+] Критерий Манна-Уитни-Уилкоксона
- t-критерий Стьюдента

**Комментарий**

Число кликов на пользователя распределено логнормально, если его визуализировать. Так что лучше применить непараметрические критерии, как например, критерий Манна-Уитни-Уилкоксона.

### 12

Эти две метрики по смыслу довольно похожи и, скорее всего, коррелируют. Так что давайте по ним ответим на вопрос "стали люди ставить больше лайков в тестовой группе". Используем уровень значимости 0.05.

Какие выводы можно сделать про изменение этих двух метрик?

- Спорно: одна из метрик улучшилась, а вторая ухудшилась
- Обе метрики статистически значимо не изменились, то есть нельзя сказать, что лайков стало больше
- [+] Обе метрики статистически значимо улучшились, то есть лайкать стали больше

**Решение**

In [13]:
from scipy.stats import mannwhitneyu

mannwhitneyu(
    tmp[tmp.exp_group == 'control'].like_num,
    tmp[tmp.exp_group == 'test'].like_num,
)

MannwhitneyuResult(statistic=518358073.0, pvalue=2.9585062792441964e-05)

Видим, что изменения значимые

### 13

А теперь давайте сделаем самое сложное — посчитаем hitrate (или долю рекомендаций, в которые пользователи кликнули). Для этого мы хотим собрать в одной табличке информацию о показанных постах и соответствующих кликах.

У нас нет айдишника, по которому мы можем сджойнить, но есть два других способа сделать это:

- По каждому пользователю отсортировать в обратном по времени порядке все лайки и показы рекомендаций (удобнее прямо в одном списке). Далее в цикле пробежаться по этому списку. Если мы встречаем лайк, то запоминаем его (лайков может быть несколько подряд, надо сохранить все). Если мы встречаем показ рекомендаций, то сохраняем его вместе с уже сохранёнными лайками (так как эти лайки следуют за этим показом рекомендаций в "обычном направлении времени"), при этом список сохранёных лайков очищаем (мы уже приписали эти лайки другим показам). Тем самым мы соединим все показы рекомендаций со всеми лайками после них. Нужно только ещё следить за тем, что может быть лайк, который случился через большое время после показа рекомендаций — скорее всего, мы потеряли другой показ рекомендаций, поэтому такой лайк не надо учитывать.
- Сделать join по пользователям всех лайков и всех показов рекомендаций (то есть у нас появятся пары всех показов и всех лайков одного и того же пользователя, но не все эти пары реальны). Далее отфильтровать те ситуации, где лайк предшествовал показу рекомендаций (то есть относился на самом деле к другому показу) или был слишком поздно (тоже относится к другому показу).

Оба эти способа не идеальные и довольно муторные. В pandas проще сделать второй из них. Попробуйте сделать его. В качестве проверки "лайк произошёл слишком поздно после рекомендации" возьмите отсечку 1 час (только лучше сначала просто помечать такие показы, как "отсутствие лайка", потому что если совсем удалять, то можно потерять показ). Добавьте также фильтрацию на то, что в рекомендациях был пост, на который пользователь лайкнул (иначе это точно был другой показ). В самом конце проведите уникализацию всех этих пар в строки показ-лайки (количество лайков может быть нулевым).

На каждом этапе фильтрации следите за размерами таблиц, это поможет вам убедиться, что происходит что-то корректное. Не забудьте учесть тот факт, что могут быть показы рекомендаций без лайков (либо через тип join, либо добавить эти строки в конце). В итоге число строк должно быть похожим на то, которые было сначала в табличке с показами.

Посчитайте долю показов, в которых был хотя бы один лайк.

- 14%
- [+] 71%
- 89%
- 19%

**Решение**

Соединим все пары показом и кликов пользователей (не забудем про показы без кликов)

In [14]:
tmp = pd.merge(views, likes, on='user_id', how='outer')
tmp.post_id = tmp.post_id.fillna(-1).astype(int)

Распарсим сами показы

In [15]:
tmp['recommendations'] = tmp.recommendations.apply(
    lambda x: list(map(int, filter(bool, x[1:-1].split(' '))))
)

Пробежимся по всем показам и сравним с лайками. Пометим то, слишком старое или из будущего, лайки по непоказанным рекомендациям

In [16]:
tmp.post_id = tmp.apply(
    lambda row:
    -1
    if
        (row.post_id == -1) | 
        ((row.timestamp_x > row.timestamp_y) |
         (row.timestamp_x + 60 * 60 < row.timestamp_y)) |
        (row.post_id not in row.recommendations)
    else
    row.post_id, axis=1)

Сравним размерности (предполагаем, что таймстемпы показов у пользователей были уникальны)

In [17]:
len(views), len(tmp)

(193268, 1016889)

In [18]:
views.groupby(['user_id', 'timestamp']).first().shape

(193268, 2)

Кажется, что ок. Давайте уникализируем

In [19]:
def my_agg(values):
    values = set(values)
    if -1 in values and len(values) >= 2:
        return 1
    elif -1 not in values:
        return 1
    return 0

tmp_agg = tmp.groupby(['user_id', 'exp_group', 'timestamp_x']).post_id.agg(my_agg)

А теперь посчитаем ответ на наш вопрос

In [20]:
tmp_agg.reset_index().post_id.mean()

0.7132582734855227

### 14

А теперь давайте оценим различие между группами и значимость. z-критерий для долей мы здесь не можем применять, так как у нас в каждой выборке один и тот же пользователь может встречаться несколько раз. Давайте применим бакетный подход (то есть перейдём к бакетам и по ним оценим значимость), чтобы поститать групповой hitrate (или CTR) — доля hitrate по группе/бакету. Используйте 100 бакетов. Уровень значимости останется тем же на уровне 0.05.

Какой вывод мы можем сделать?

- [+] В тестовой группе hitrate выше на 1 п.п. (до бакетирования), есть статистически значимая разница
- Нет статистически значимой разницы между hitrate в группах
- В тестовой группе hitrate ниже на 2 п.п. (до бакетирования), есть статистически значимая разница
- В тестовой группе hitrate выше на 3 п.п. (до бакетирования), есть статистически значимая разница

**Решение**

In [21]:
tmp_agg = tmp_agg.reset_index().rename(columns={'post_id': 'hitrate'})

Подготовим бакеты

In [22]:
import hashlib

tmp_agg['bucket'] = tmp_agg['user_id'].apply(
    lambda x: int(hashlib.md5((str(x) + 'bbb').encode()).hexdigest(), 16) % 100
)

In [23]:
tmp_agg['view'] = 1

new_df = tmp_agg.groupby(['exp_group', 'bucket']).sum(['hitrate', 'view']).reset_index()
new_df['hitrate_new'] = new_df.hitrate / new_df.view

new_df.head()

,exp_group,bucket,user_id,timestamp_x,hitrate,view,hitrate_new
0,control,0,75055655,1472627637383,621,890,0.697753
1,control,1,78264361,1557009869357,668,941,0.709883
2,control,2,89756485,1664574067853,707,1006,0.702783
3,control,3,74901253,1523924072519,646,921,0.701412
4,control,4,77518818,1603347973264,710,969,0.732714


Посчитаем метрику

In [24]:
tmp_agg.groupby('exp_group').hitrate.mean()

exp_group
control    0.707741
test       0.720975
Name: hitrate, dtype: float64

Разница довольно большая! Но что со значимостью

In [25]:
from scipy.stats import mannwhitneyu, ttest_ind

In [26]:
mannwhitneyu(
    new_df[new_df.exp_group == 'control'].hitrate_new,
    new_df[new_df.exp_group == 'test'].hitrate_new,
)

MannwhitneyuResult(statistic=2452.0, pvalue=4.829847062588435e-10)

In [27]:
ttest_ind(
    new_df[new_df.exp_group == 'control'].hitrate_new,
    new_df[new_df.exp_group == 'test'].hitrate_new,
)

Ttest_indResult(statistic=-6.304578169376004, pvalue=1.8432228173570576e-09)

Изменение значимое! Метрика улучшилась

### 15

Вот мы и провели с вами A/B эксперимент с моделями, а также проанализировали его. Мы ответили на самый главный вопрос  (стали ли наши рекомендации лучше), но в реальном продукте мо